On this Section I set a new goal of test and it is gender(more correctly vocal of the speaker). Although below audios are generated with Text to speach model, but the reasoning of this test to check model performance on different vocals.

TEST DATA contains- 3 type of boys and 3 type of girls audios on 2 Type of texts: formal, informal

TEST DATA FOLDER - AG_tests


AG data generated using this website https://www.narakeet.com/

In [1]:
import whisper
import glob
import jiwer
import pandas as pd
import os

In [2]:
model = whisper.load_model('medium')

In [3]:
def calc_wer(clean_text, hypotes, punct_marks=False, filler_words=True):
    # filler words used in language to spent time while thinking on something or just to make noise -_-
    # removing them might help to clearly find WER on to texts
    filler_words = ["oh", "um", "uh", "er", "ah", "like", "well", "so", "right", "literally", "okay"]
    clean_text = clean_text.strip().lower()
    hypotes = hypotes.strip().lower()    
    if punct_marks:
        for mark in (',', '-', ':', ';', '?', '!', "'"):
            clean_text = clean_text.replace(mark, '')
            hypotes = hypotes.replace(mark, '')
        clean_text = clean_text.replace('...', ' ')
        hypotes = hypotes.replace('...', ' ')
    if filler_words:
        for fil in filler_words:
            clean_text = clean_text.replace(fil, ' ')
            hypotes = hypotes.replace(fil, ' ')
    return jiwer.wer(clean_text, hypotes)

In [4]:
def preprocess_and_create_df():
    """This function creates DataFrame object out of environment dataset located on AG_tests folder"""
    dir_path = "AG_tests/"
    all_dfs = []
    for fold in os.listdir(dir_path):
        if not os.path.isdir(f"{dir_path}{fold}"):
            continue
        with open(f'{dir_path}{fold}/text.txt', 'rt') as f:
            text = f.read().strip()
        df = pd.DataFrame()
        df['audio_paths'] = sorted(glob.glob(f"{dir_path}{fold}/*.wav"))
        df['name'] = df['audio_paths'].apply(lambda x: x.split('/')[-1].split('.')[0])
        df['gender'] = df['name'].apply(lambda x: 1 if 'boy' in x else 0)
        df['clea_text'] = text
        df['text_type'] = fold.split('_')[0]
        all_dfs.append(df)
    return pd.concat(all_dfs)

In [5]:
test_df = preprocess_and_create_df()

In [6]:
test_df

,audio_paths,name,gender,clea_text,text_type
0,AG_tests/formal_text_2/boy_1.wav,boy_1,1,"To Whom It May Concern, I am writing to reques...",formal
1,AG_tests/formal_text_2/boy_2.wav,boy_2,1,"To Whom It May Concern, I am writing to reques...",formal
2,AG_tests/formal_text_2/boy_3.wav,boy_3,1,"To Whom It May Concern, I am writing to reques...",formal
3,AG_tests/formal_text_2/girl_1.wav,girl_1,0,"To Whom It May Concern, I am writing to reques...",formal
4,AG_tests/formal_text_2/girl_2.wav,girl_2,0,"To Whom It May Concern, I am writing to reques...",formal
5,AG_tests/formal_text_2/girl_3.wav,girl_3,0,"To Whom It May Concern, I am writing to reques...",formal
0,AG_tests/informal_text_1/boy_1.wav,boy_1,1,"Hey, what’s up? How’s it going? It’s been a mi...",informal
1,AG_tests/informal_text_1/boy_2.wav,boy_2,1,"Hey, what’s up? How’s it going? It’s been a mi...",informal
2,AG_tests/informal_text_1/boy_3.wav,boy_3,1,"Hey, what’s up? How’s it going? It’s been a mi...",informal
3,AG_tests/informal_text_1/girl_1.wav,girl_1,0,"Hey, what’s up? How’s it going? It’s been a mi...",informal


In [7]:
test_df['hypotes_med'] = test_df['audio_paths'].apply(lambda x: model.transcribe(x)['text'])

In [8]:
test_df['medium_WER_with_mark'] = [calc_wer(row['clea_text'], row['hypotes_med']) for i, row in test_df.iterrows()] 

test_df['medium_WER_no_mark'] = [calc_wer(row['clea_text'], row['hypotes_med'], punct_marks=True) for i, row in test_df.iterrows()] 

Below shown some results after grouping to some categories

In [9]:
test_df.groupby('gender')[['medium_WER_no_mark', 'medium_WER_with_mark']].mean()

,medium_WER_no_mark,medium_WER_with_mark
gender,,
0,0.048868,0.052033
1,0.050002,0.055458


ON gender department results show that models performs somewhat equally(worth mentioning that this dataset is a clean dataset) a bit tilted to better understand girls speach but difference is not big. Per my understanding I need to check on bigger and wider range of dataset this feature cuz I think there should be some different result for different vocals.

In [10]:
test_df.groupby('text_type')[['medium_WER_no_mark', 'medium_WER_with_mark']].mean()

,medium_WER_no_mark,medium_WER_with_mark
text_type,,
formal,0.029359,0.033495
informal,0.069511,0.073996


On formal and informal text tho model performs differently, my understanding on this that train data of the model supervised mainly on formal text, so I think in daily basis meetings or talks this model not in a good shape

Here will be tested accent versus no accent difference again on AG data

Test data contains 3 normal text on indian accent and on american english(per my understanding american english is the one without accent as it is easy to understand for a wider range of listeners)

Test data folder AG_test_accent

In [44]:
def preprocess_test_data():
    dir_path = 'AG_test_accent/'
    df = pd.DataFrame()
    df['american_audio'] = sorted(glob.glob(f'{dir_path}american/*.wav'))
    df['indian_audio'] = sorted(glob.glob(f'{dir_path}indian/*.wav'))
    texts = []
    for txt in sorted(glob.glob(f'{dir_path}texts/*.txt')):
        with open(txt) as f:
            texts.append(f.read().replace('\\', '').strip())
    df['clean_text'] = texts
    return df

In [45]:
acc_test = preprocess_test_data()

In [46]:
acc_test

,american_audio,indian_audio,clean_text
0,AG_test_accent/american/t1.wav,AG_test_accent/indian/t1.wav,I am a marine biologist and today was a very e...
1,AG_test_accent/american/t2.wav,AG_test_accent/indian/t2.wav,"As a software engineer, my workday is usually ..."
2,AG_test_accent/american/t3.wav,AG_test_accent/indian/t3.wav,I'm a high school English teacher and today we...


In [47]:
acc_test['am_hypotes'] = acc_test['american_audio'].apply(lambda x: model.transcribe(x)['text'])

In [48]:
acc_test['ind_hypotes'] = acc_test['indian_audio'].apply(lambda x: model.transcribe(x)['text'])

In [49]:
acc_test['am_medium_WER_with_mark'] = [calc_wer(row['clean_text'], row['am_hypotes']) for i, row in acc_test.iterrows()] 

acc_test['am_medium_WER_no_mark'] = [calc_wer(row['clean_text'], row['am_hypotes'], punct_marks=True) for i, row in acc_test.iterrows()] 

acc_test['ind_medium_WER_with_mark'] = [calc_wer(row['clean_text'], row['ind_hypotes']) for i, row in acc_test.iterrows()] 

acc_test['ind_medium_WER_no_mark'] = [calc_wer(row['clean_text'], row['ind_hypotes'], punct_marks=True) for i, row in acc_test.iterrows()] 

In [53]:
acc_test

,american_audio,indian_audio,clean_text,am_hypotes,ind_hypotes,am_medium_WER_with_mark,am_medium_WER_no_mark,ind_medium_WER_with_mark,ind_medium_WER_no_mark
0,AG_test_accent/american/t1.wav,AG_test_accent/indian/t1.wav,I am a marine biologist and today was a very e...,I am a marine biologist and today was a very ...,I am a marine biologist and today was a very ...,0.000000,0.0,0.000000,0.000000
1,AG_test_accent/american/t2.wav,AG_test_accent/indian/t2.wav,"As a software engineer, my workday is usually ...","As a software engineer, my workday is usually...","As a software engineer, my workday is usually...",0.000000,0.0,0.000000,0.000000
2,AG_test_accent/american/t3.wav,AG_test_accent/indian/t3.wav,I'm a high school English teacher and today we...,I'm a high school English teacher and today w...,I am a high school English teacher and today ...,0.033708,0.0,0.101124,0.068182


Sadly my test data was bad I guess it didn't find any valuable differences on indian accent , last row result just shows that model doesnt handle titles such as book title and couldnt convert them into text with "" symbols

My next test is on science fields in this case I take one biology text and it text 

In [70]:
def preprocess_field_data():
    data_dir = 'AG_test_science_fields/'
    df = pd.DataFrame()
    df['biolog_audio'] = [f"{data_dir}biology.wav"]
    df['it_audio'] = f"{data_dir}it.wav"
    with open(f"{data_dir}biology.txt") as f:
        df['biolog_txt'] = f.read().strip()
    with open(f"{data_dir}it.txt" ) as f:
        df['it_txt'] = f.read().strip()
    return df

In [78]:
field_test = preprocess_field_data()

In [79]:
field_test

,biolog_audio,it_audio,biolog_txt,it_txt
0,AG_test_science_fields/biology.wav,AG_test_science_fields/it.wav,"At the cellular level, biology explores the co...",One of the key areas of focus in IT is softwar...


In [80]:
field_test['biology_hypotes'] = field_test['biolog_audio'].apply(lambda x: model.transcribe(x)['text'])
field_test['it_hypotes'] = field_test['it_audio'].apply(lambda x: model.transcribe(x)['text'])


In [81]:
field_test['biology_medium_WER_with_mark'] = [calc_wer(row['biolog_txt'], row['biology_hypotes']) for i, row in field_test.iterrows()] 

field_test['it_medium_WER_no_mark'] = [calc_wer(row['it_txt'], row['it_hypotes'], punct_marks=True) for i, row in field_test.iterrows()] 


In [82]:
field_test 

,biolog_audio,it_audio,biolog_txt,it_txt,biology_hypotes,it_hypotes,biology_medium_WER_with_mark,it_medium_WER_no_mark
0,AG_test_science_fields/biology.wav,AG_test_science_fields/it.wav,"At the cellular level, biology explores the co...",One of the key areas of focus in IT is softwar...,"At the cellular level, biology explores the c...",One of the key areas of focus in IT is softwa...,0.0,0.008264


Although test shows model can understand different field terminology, but I think with bigger and wider range of data there is a possibility to find some part of fields that this model is not trained of and can not detect clearly